<a href="https://colab.research.google.com/github/IveySong/MNE_Tutorial/blob/main/Visualizing_EEG_with_MNE_(ICA%2C_topomaps%2C_filters).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nnair25_alcoholics_path = kagglehub.dataset_download('nnair25/Alcoholics')

print('Data source import complete.')


# MNE Tutorial: Different ways of visualizing an EEG signal
This notebook will teach you how to use the MNE library to plot EEG data.

In [ ]:
import mne # MNE is already installed in the kaggle image. pip install is not required.

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import glob
from matplotlib import pyplot as plt

# Loading the dataset
This dataset comes in separate csv files. We will concat them into a single Pandas DataFrame.

In [ ]:
_dfs_list = []
for csv_filename in tqdm(glob.glob('../input/Alcoholics/SMNI_CMI_TRAIN/*.csv')):
    _dfs_list.append(pd.read_csv(csv_filename))
df = pd.concat(_dfs_list)
del(_dfs_list)
df = df.drop(['Unnamed: 0'], axis=1)
df.head(3)

In [ ]:
# correct the 'sensor position' names

channel_list = list(set(df['sensor position']))
channel_list.sort()

channel_mapping = {
    'AFZ':'AFz',
    'CPZ':'CPz',
    'CZ':'Cz',
    'FCZ':'FCz',
    'FP1':'Fp1',
    'FP2':'Fp2',
    'FPZ':'Fpz',
    'FZ':'Fz',
    'OZ':'Oz',
    'POZ':'POz',
    'PZ':'Pz',
}

channel_mapping_full = dict()

for ch in channel_list:
    if ch in channel_mapping:
        channel_mapping_full[ch] = channel_mapping[ch]
    else:
        channel_mapping_full[ch] = ch

channel_list_fixed = [channel_mapping_full[ch] for ch in channel_list]

df['sensor position'] = df['sensor position'].map(channel_mapping_full)
df.head(3)

In [ ]:
# transpose the table to make the data extraction easier

transposed_df_list = []

for group_df in tqdm(df.groupby(['name', 'trial number', 'matching condition', 'sensor position', 'subject identifier'])):
    _df = pd.DataFrame(group_df[1]['sensor value']).T
    _df.columns = [f'sample_{idx}' for idx in range(256)]
    _df['name'] = group_df[0][0]
    _df['trial number'] = group_df[0][1]
    _df['matching condition'] = group_df[0][2]
    _df['sensor position'] = group_df[0][3]
    _df['subject identifier'] = group_df[0][4]

    transposed_df_list.append(_df)

df = pd.concat(transposed_df_list)
df = df[[*df.columns[-5:],*df.columns[0:-5]]]
df = df.reset_index(drop=True)
df.head(3)

# Displaying raw signal as an image
You can use the images for CNN training

In [ ]:
def get_record_df(df, name, trial_number, matching_condition, channel_list):
    df_record = df[df['name'].eq(name) & df['trial number'].eq(trial_number) & df['matching condition'].eq(matching_condition)].set_index(['sensor position']).loc[channel_list]
    return df_record

df_record = get_record_df(df, 'co2a0000364', 0, 'S1 obj', channel_list_fixed)
df_record

In [ ]:
def get_signal_array(df, name, trial_number, matching_condition, channel_list):
    df_record = get_record_df(df, name, trial_number, matching_condition, channel_list)
    return df_record.to_numpy()[:, 4:]

signal_array = get_signal_array(df, 'co2a0000364', 10, 'S1 obj', channel_list_fixed)
signal_array.shape

In [ ]:
plt.title('Signal array as an image (64 x 256)')
plt.ylabel('Channels (sensor position)');
plt.xlabel('Sample number');
plt.imshow(signal_array.astype(int));
plt.show()

channels_to_display = ['AF1', 'CP3', 'F1']
for channel in channels_to_display:
    plt.xlabel('Sample number');
    plt.plot(signal_array[channel_list.index(channel)]);
plt.legend(channels_to_display);

# Load signal data to an MNE Raw object
Create Info object. input = channel list, data freq, channel types

Create Raw object. input = the signal array and the Info object

Set Raw object montage. After we attach a montage to the signal, we can display it in a topmap.

In [ ]:
info = mne.create_info(ch_names=channel_list_fixed, sfreq=256, ch_types=['eeg']*64)
raw = mne.io.RawArray(signal_array, info)

standard_1020_montage = mne.channels.make_standard_montage('standard_1020');
raw.drop_channels(['X', 'Y', 'nd'])
raw.set_montage(standard_1020_montage)

# Plot freqs

In [ ]:
raw.plot_psd();
raw.plot_psd(average=True);

# Apply filter

In [ ]:
raw_filtered = raw.copy().filter(8,30, verbose=False);
raw_filtered.plot_psd();
raw_filtered.plot_psd(average=True);

In [ ]:
plt.imshow(raw.get_data())
plt.show()
plt.imshow(raw.copy().filter(1,10, verbose=False).get_data());
plt.show()
plt.plot(raw.copy().get_data()[40])
plt.plot(raw.copy().filter(8,30, verbose=False).get_data()[40])

# ICA

In [ ]:
ica = mne.preprocessing.ICA(random_state=42, n_components=20)
ica.fit(raw.copy().filter(1,None, verbose=False), verbose=False)
ica.plot_components()

# Plot a topographic map as an image

In [ ]:
def plot_eeg_topomap(signal_array, save_path_animation=None, show_names=False, start_time=0.05, end_time=1, step_size=0.1):
    # select channel X,Y format
    montage = mne.channels.make_standard_montage('standard_1020')

    # remove channels that don't have X,Y positions
    ch_to_remove = []
    for ch in channel_list_fixed:
        if ch not in list(set(montage.ch_names).intersection(channel_list_fixed)):
            ch_to_remove.append(channel_list_fixed.index(ch))
    arr = np.delete(signal_array.copy(), ch_to_remove, axis=0)

    # create info+evoked objects
    info = mne.create_info(ch_names=list(set(montage.ch_names).intersection(channel_list_fixed)), sfreq=256, ch_types='eeg')
    evoked = mne.EvokedArray(arr, info)

    # set channel X,Y positions
    evoked.set_montage(montage)

    # plot img
    evoked.plot_topomap(np.arange(start_time, end_time, step_size), ch_type='eeg', time_unit='s', ncols=5, nrows=2, show_names=show_names);

In [ ]:
plot_eeg_topomap(signal_array, show_names=False)

# Plot a topographic map as an animation

In [ ]:
def plot_eeg_topomap_animation(signal_array, save_path_animation, start_time=0.00, end_time=1, step_size=0.01, frame_rate=10):
    # select channel X,Y format
    montage = mne.channels.make_standard_montage('standard_1020')

    # remove channels that don't have X,Y positions
    ch_to_remove = []
    for ch in channel_list_fixed:
        if ch not in list(set(montage.ch_names).intersection(channel_list_fixed)):
            ch_to_remove.append(channel_list_fixed.index(ch))
    arr = np.delete(signal_array.copy(), ch_to_remove, axis=0)

    # create info+evoked objects
    info = mne.create_info(ch_names=list(set(montage.ch_names).intersection(channel_list_fixed)), sfreq=256, ch_types='eeg')
    evoked = mne.EvokedArray(arr, info)

    # set channel X,Y positions
    evoked.set_montage(montage)

    # (optional) plot and save animation
    if save_path_animation:
        fig, anim = evoked.animate_topomap(times=np.arange(start_time, end_time, step_size), frame_rate=frame_rate, butterfly=True, blit=False, );
        anim.save(save_path_animation);
        print('saved to', save_path_animation)

In [ ]:
plot_eeg_topomap_animation(signal_array, './gif.gif')

![](./gif.gif)

In [ ]:
def plot_eeg_joint_topomap(signal_array, save_path_animation=None, show_names=False, start_time=0.05, end_time=1, step_size=0.1):
    # select channel X,Y format
    montage = mne.channels.make_standard_montage('standard_1020')

    # remove channels that don't have X,Y positions
    ch_to_remove = []
    for ch in channel_list_fixed:
        if ch not in list(set(montage.ch_names).intersection(channel_list_fixed)):
            ch_to_remove.append(channel_list_fixed.index(ch))
    arr = np.delete(signal_array.copy(), ch_to_remove, axis=0)

    # create info+evoked objects
    info = mne.create_info(ch_names=list(set(montage.ch_names).intersection(channel_list_fixed)), sfreq=256, ch_types='eeg')
    evoked = mne.EvokedArray(arr, info)

    # set channel X,Y positions
    evoked.set_montage(montage)

    # plot img
    evoked.plot_joint();

plot_eeg_joint_topomap(signal_array)

In [ ]:
#https://www.youtube.com/watch?v=wNIaT1UT6rI&list=PLXtvZiGkmNVvPS0N9UNBVkIFe0_0t_Nqt&index=3